In [1]:
import mlflow
import pickle
import sys
sys.path.insert(1, '../library')
import database_helper
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


2024-07-30 21:42:19.339458: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 21:42:19.339815: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 21:42:19.341574: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 21:42:19.346553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 21:42:19.354807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
os.environ["MLFLOW_TRACKING_USERNAME"] = "user"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "password"
mlflow.set_tracking_uri(uri="http://mlflow.local:80")
mlflow.set_experiment("DWD Autoencoder Anomaly Detection")
mlflow.autolog()

2024/07/30 21:42:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/07/30 21:42:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/07/30 21:42:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


In [3]:
with open('selected_columns.pkl', 'rb') as f:
    selected_columns = pickle.load(f)
dataframe = database_helper.query_data(field_list=selected_columns)
dataframe['_time'] = pd.to_datetime(dataframe['_time'])
dataframe.set_index('_time', inplace=True)
dataframe = dataframe.select_dtypes(include='float64')
dataframe.interpolate(inplace=True)
display(dataframe)

,10838_days_0_precipitation,10838_days_0_temperatureMax,10838_days_0_windDirection,10838_days_1_sunshine,10838_days_1_temperatureMin,10838_days_1_windDirection,10838_days_2_precipitation,10838_days_2_sunshine,10838_days_2_temperatureMax,10838_days_2_temperatureMin,...,10840_days_7_windDirection,10840_days_8_sunshine,10840_days_8_temperatureMax,10840_days_8_temperatureMin,10840_days_8_windDirection,10840_days_8_windGust,10840_days_9_precipitation,10840_days_9_sunshine,10840_days_9_temperatureMax,10840_days_9_windDirection
_time,,,,,,,,,,,,,,,,,,,,,
2024-07-28 12:00:00+00:00,59.0,229.0,3210.0,8200.0,119.0,720.0,0.0,7590.0,303.0,123.0,...,2580.0,4360.0,236.0,151.0,2470.0,315.0,0.0,4280.0,246.0,2380.0
2024-07-28 12:00:10+00:00,59.0,229.0,3210.0,8200.0,119.0,720.0,0.0,7590.0,303.0,123.0,...,2580.0,4360.0,236.0,151.0,2470.0,315.0,0.0,4280.0,246.0,2380.0
2024-07-28 12:00:20+00:00,59.0,229.0,3210.0,8200.0,119.0,720.0,0.0,7590.0,303.0,123.0,...,2580.0,4360.0,236.0,151.0,2470.0,315.0,0.0,4280.0,246.0,2380.0
2024-07-28 12:00:30+00:00,59.0,229.0,3210.0,8200.0,119.0,720.0,0.0,7590.0,303.0,123.0,...,2580.0,4360.0,236.0,151.0,2470.0,315.0,0.0,4280.0,246.0,2380.0
2024-07-28 12:00:40+00:00,59.0,229.0,3210.0,8200.0,119.0,720.0,0.0,7590.0,303.0,123.0,...,2580.0,4360.0,236.0,151.0,2470.0,315.0,0.0,4280.0,246.0,2380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-30 19:41:40+00:00,16.0,303.0,1720.0,6810.0,184.0,2440.0,129.0,2860.0,248.0,186.0,...,2210.0,4630.0,248.0,159.0,2400.0,296.0,0.0,4370.0,245.0,2660.0
2024-07-30 19:41:50+00:00,16.0,303.0,1720.0,6810.0,184.0,2440.0,129.0,2860.0,248.0,186.0,...,2210.0,4630.0,248.0,159.0,2400.0,296.0,0.0,4370.0,245.0,2660.0
2024-07-30 19:42:00+00:00,16.0,303.0,1720.0,6810.0,184.0,2440.0,129.0,2860.0,248.0,186.0,...,2210.0,4630.0,248.0,159.0,2400.0,296.0,0.0,4370.0,245.0,2660.0


In [4]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(dataframe)
display(data_scaled)

array([[ 1.92834816, -1.51974948,  1.80598864, ..., -1.85872586,
         0.28820654, -1.87897217],
       [ 1.92834816, -1.51974948,  1.80598864, ..., -1.85872586,
         0.28820654, -1.87897217],
       [ 1.92834816, -1.51974948,  1.80598864, ..., -1.85872586,
         0.28820654, -1.87897217],
       ...,
       [-0.02841177,  1.06627722,  0.13155439, ..., -1.19833616,
        -0.10309542,  1.01254355],
       [-0.02841177,  1.06627722,  0.13155439, ..., -1.19833616,
        -0.10309542,  1.01254355],
       [-0.02841177,  1.06627722,  0.13155439, ..., -1.19833616,
        -0.10309542,  1.01254355]])

In [5]:
train_data, test_data = train_test_split(data_scaled, test_size=0.2) #, random_state=42
input_dim = train_data.shape[1]
encoding_dim = 128

input_layer = Input(shape=(input_dim,))
encoder = Dense(512, activation="relu")(input_layer)
encoder = Dense(256, activation="relu")(encoder)
encoder = Dense(128, activation="relu")(encoder)
encoder = Dense(encoding_dim, activation="relu")(encoder)

decoder = Dense(128, activation="relu")(encoder)
decoder = Dense(256, activation="relu")(decoder)
decoder = Dense(512, activation="relu")(decoder)
decoder = Dense(input_dim, activation="sigmoid")(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
display(autoencoder.summary())

optimizer = Adam(learning_rate=0.0001)
autoencoder.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'accuracy'])
# Train the autoencoder
autoencoder.fit(train_data, train_data, 
                epochs=150, 
                batch_size=32, 
                validation_split=0.2, 
                verbose=1,
                callbacks=[early_stopping])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        49,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 96)             │        49,248 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 460,768 (1.76 MB)

 Trainable params: 460,768 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

None

2024/07/30 21:42:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '45a3f23e85e245dcb83cd4d19b6ad98f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current keras workflow


Epoch 1/150
401/401 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 0.8612 - mse: 0.8612 - val_accuracy: 0.0000e+00 - val_loss: 0.5771 - val_mse: 0.5771
Epoch 2/150
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 0.5842 - mse: 0.5842 - val_accuracy: 0.0000e+00 - val_loss: 0.5733 - val_mse: 0.5733
Epoch 3/150
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 9.5728e-04 - loss: 0.5863 - mse: 0.5863 - val_accuracy: 0.1125 - val_loss: 0.5721 - val_mse: 0.5721
Epoch 4/150
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1009 - loss: 0.5886 - mse: 0.5886 - val_accuracy: 0.1125 - val_loss: 0.5721 - val_mse: 0.5721
Epoch 5/150
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1022 - loss: 0.5845 - mse: 0.5845 - val_accuracy: 0.1125 - val_loss: 0.5721 - val_mse: 0.5721
Epoch 6/150
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1032 - loss: 0.5838 - mse: 0.5838 - val_accuracy: 0.1125 - val_loss: 0.5716 - val_mse: 0.5716
Epoch 7/150
40

2024/07/30 21:42:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run big-fly-347 at: http://mlflow.local:80/#/experiments/2/runs/45a3f23e85e245dcb83cd4d19b6ad98f.
2024/07/30 21:42:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow.local:80/#/experiments/2.


In [6]:
# Use the autoencoder to reconstruct the test data
reconstructed_data = autoencoder.predict(test_data)

# Calculate reconstruction error
reconstruction_error = np.mean(np.square(test_data - reconstructed_data), axis=1)

# Define a threshold for anomaly detection (this is a simple way, more advanced methods can be used)
threshold = np.percentile(reconstruction_error, 95)

# Identify anomalies
anomalies = reconstruction_error > threshold

# Print results
print("Number of anomalies detected:", np.sum(anomalies))

126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step
Number of anomalies detected: 142
